In [65]:
import pandas as pd
import numpy as np
fnm2109 = pd.read_csv('FNM_MLLD_202109.txt', sep="|")
cpr2109_2110 = pd.read_csv('CPR_Calculation_202109-202110.csv')

/Users/qimingfeng/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (26,29,62,63,66,71) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
#fnm2109.groupby("Property State").size().sort_values()

In [66]:
fnm2110 = pd.read_csv('FNM_MLLD_202110.txt', sep="|")
cpr2110_2111 = pd.read_csv('CPR_Calculation_202110-202111.csv')

/Users/qimingfeng/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (63,66,71) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [67]:
fnm2111 = pd.read_csv('FNM_MLLD_202111.txt', sep="|")
cpr2111_2112 = pd.read_csv('CPR_Calculation_202111-202112.csv')

In [68]:
fnm2112 = pd.read_csv('FNM_MLLD_202112.txt', sep="|")
cpr2112_2201 = pd.read_csv("CPR_Calculation_202112-202201.csv")

/Users/qimingfeng/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (63,71) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [69]:
fnm2201 = pd.read_csv('FNM_MLLD_202201.txt', sep="|")
cpr2201_2202 = pd.read_csv("CPR_Calculation_202201-202202.csv")

In [70]:
def get_data(data, cpr):
    data = data.merge(cpr, on='CUSIP')
    data = data.dropna(axis=1, how='all', thresh=None, subset=None, inplace=False)
    data['First Time Home Buyer Indicator'] = data['First Time Home Buyer Indicator'].replace(['Y','N'],['Yes','No'])
    data['Loan Purpose'] = data['Loan Purpose'].replace(['C','R','P','U'],['Cash-Out','Refinance','Purchase',
                                                                           'Not specified'])
    data['Occupancy Status'] = data['Occupancy Status'].replace(['P','S','I','U'],['Principal','Second','Investor',
                                                                                   'Unknown'])
    data['Channel'] = data['Channel'].replace(['R','C','B'],['Retail','Correspondent','Broker'])
    
    state = ["NY", "TX", "CA", "IL", "FL"]
    data = data[data["Property State"].isin(state)]

    cat_variable = ['Property State','First Time Home Buyer Indicator','Loan Purpose','Occupancy Status',
               'Channel']
    data_need1=pd.DataFrame()
    alist = []
    for i in data["CUSIP"].unique():
        alist.append(i)
    for i in alist:
        subdata = data[data['CUSIP'] == i]
        df = pd.DataFrame({'CUSIP':[i]})
        for x in cat_variable:
            a = subdata.groupby(x)[ 'Loan Identifier'].count()
            a = a.to_frame().transpose().reset_index(drop=True)
            a = a/len(subdata)
            df = df.join(a)
        data_need1=data_need1.append(df,ignore_index=True)
    
    num_variable = ['Original Interest Rate','Loan Term','Loan Age','Loan-To-Value (LTV)','Debt-To-Income (DTI)',
               'Borrower Credit Score','Number of Borrowers','Mortgage Insurance Percent', "SMM", "CPR"]
    data_need2=pd.DataFrame()
    for i in alist:
        subdata = data[data['CUSIP'] == i]
        df = pd.DataFrame({'CUSIP':[i]})
        for i in num_variable:
            value = subdata[i].mean()
            b = pd.DataFrame({i:[value]})
            df = df.join(b)
        data_need2=data_need2.append(df,ignore_index=True)
        
    data_need3 = pd.merge(data_need1, data_need2, on="CUSIP")
    return data_need3
    

In [84]:
data2109_2110 = get_data(fnm2109,cpr2109_2110)
data2109_2110
#data2109_2110.to_csv('data2109_2110.csv')

,CUSIP,CA,FL,IL,NY,TX,No,Yes,Cash-Out,N,Purchase,Investor,Principal,Second,Broker,Correspondent,Retail,Original Interest Rate,Loan Term,Loan Age,Loan-To-Value (LTV),Debt-To-Income (DTI),Borrower Credit Score,Number of Borrowers,Mortgage Insurance Percent,SMM,CPR
0,31418D3X8,0.772761,0.085457,0.042700,0.019155,0.079927,0.866883,0.133117,0.321247,0.420729,0.258024,0.005188,0.975942,0.018870,0.465025,0.188188,0.346788,2.872365,360.0,0.973662,63.298615,36.416681,759.119720,1.475058,3.630751,0.003131,0.036937
1,31418D3Y6,0.699621,0.088109,0.085910,0.036417,0.089943,0.840401,0.159599,0.477209,0.253330,0.269461,0.013565,0.962483,0.023952,0.195283,0.185995,0.618722,3.310758,360.0,1.093975,70.481364,37.470121,737.421850,1.504949,5.030062,0.005066,0.059132
2,31418D4V1,0.411765,0.352941,NaN,NaN,0.235294,1.000000,NaN,0.705882,0.058824,0.235294,0.352941,0.588235,0.058824,0.058824,0.294118,0.647059,4.875000,360.0,1.352941,70.882353,34.941176,693.882353,1.588235,2.176471,0.001423,0.016946
3,31418D4A7,0.380952,0.119048,0.107143,0.047619,0.345238,0.952381,0.047619,0.690476,0.154762,0.154762,0.130952,0.833333,0.035714,0.083333,0.226190,0.690476,4.341964,360.0,1.130952,74.380952,38.785714,689.773810,1.440476,2.809524,0.017708,0.192979
4,31418D3W0,0.813559,0.092010,0.012107,0.012107,0.070218,0.949153,0.050847,0.208232,0.690073,0.101695,NaN,0.987893,0.012107,0.498789,0.278450,0.222760,2.370646,360.0,1.106538,56.617433,34.346247,768.302663,1.472155,1.612591,0.003351,0.039477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,31418DLU4,0.563574,0.144330,0.082474,0.051546,0.158076,0.776632,0.223368,0.463918,0.130584,0.405498,0.120275,0.865979,0.013746,0.336770,0.347079,0.316151,4.869261,360.0,20.374570,75.920962,40.051546,700.807560,1.570447,5.079038,0.049439,0.455797
121,31418DL34,0.521739,0.195652,0.086957,0.032609,0.163043,0.836957,0.163043,0.532609,0.086957,0.380435,0.228261,0.771739,NaN,0.260870,0.336957,0.402174,5.340152,360.0,20.532609,78.250000,39.163043,689.336957,1.543478,5.293478,0.065214,0.554810
122,31418DLV2,0.617945,0.136035,0.063676,0.021708,0.160637,0.798842,0.201158,0.422576,0.206946,0.370478,0.073806,0.914616,0.011577,0.183792,0.536903,0.279305,4.399460,360.0,20.386397,75.643994,39.917511,727.480463,1.531114,6.752533,0.054582,0.490097
123,31418DLT7,0.718144,0.084557,0.063124,0.026424,0.107751,0.873752,0.126248,0.306518,0.395185,0.298297,0.017029,0.949501,0.033470,0.268056,0.324427,0.407516,3.868637,360.0,20.327657,70.571932,37.640341,749.679389,1.515561,5.165590,0.050817,0.465188


In [85]:
data2110_2111 = get_data(fnm2110,cpr2110_2111)
data2110_2111
#data2110_2111.to_csv('data2110_2111.csv')

,CUSIP,CA,FL,IL,NY,TX,No,Yes,Cash-Out,N,Purchase,Investor,Principal,Second,Broker,Correspondent,Retail,Original Interest Rate,Loan Term,Loan Age,Loan-To-Value (LTV),Debt-To-Income (DTI),Borrower Credit Score,Number of Borrowers,Mortgage Insurance Percent,SMM,CPR
0,31418D3X8,0.772696,0.085474,0.042766,0.019185,0.079879,0.866735,0.133265,0.321400,0.420521,0.258079,0.005196,0.976019,0.018785,0.465627,0.187793,0.346580,2.872258,360.0,1.972993,63.305812,36.417723,759.118477,1.475277,3.630810,0.001927,0.022884
1,31418D3Y6,0.699277,0.088268,0.086184,0.036533,0.089739,0.840137,0.159863,0.477136,0.253770,0.269094,0.013485,0.962731,0.023783,0.195292,0.185853,0.618855,3.310847,360.0,2.093417,70.476891,37.470516,737.322422,1.505088,5.028932,0.002581,0.030537
2,31418D4V1,0.411765,0.352941,NaN,NaN,0.235294,1.000000,NaN,0.705882,0.058824,0.235294,0.352941,0.588235,0.058824,0.058824,0.294118,0.647059,4.875000,360.0,2.352941,70.882353,34.941176,693.882353,1.588235,2.176471,0.039451,0.383076
3,31418D4A7,0.365854,0.121951,0.109756,0.048780,0.353659,0.951220,0.048780,0.695122,0.158537,0.146341,0.134146,0.841463,0.024390,0.085366,0.219512,0.695122,4.342683,360.0,2.121951,74.243902,38.658537,689.378049,1.439024,2.878049,0.001709,0.020316
4,31418D3W0,0.813559,0.092010,0.012107,0.012107,0.070218,0.949153,0.050847,0.208232,0.690073,0.101695,NaN,0.987893,0.012107,0.498789,0.278450,0.222760,2.370646,360.0,2.106538,56.617433,34.346247,768.302663,1.472155,1.583535,0.000082,0.000980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,31418DLU4,0.564748,0.143885,0.075540,0.053957,0.161871,0.769784,0.230216,0.449640,0.136691,0.413669,0.125899,0.859712,0.014388,0.330935,0.352518,0.316547,4.869155,360.0,21.381295,75.888489,40.032374,700.449640,1.568345,5.273381,0.057847,0.510834
121,31418DL34,0.541176,0.176471,0.094118,0.035294,0.152941,0.823529,0.176471,0.517647,0.070588,0.411765,0.235294,0.764706,NaN,0.258824,0.317647,0.423529,5.338753,360.0,21.505882,78.482353,39.082353,690.411765,1.529412,5.729412,0.055782,0.497813
122,31418DLV2,0.615267,0.137405,0.064122,0.022901,0.160305,0.800000,0.200000,0.432061,0.200000,0.367939,0.074809,0.912977,0.012214,0.181679,0.532824,0.285496,4.400311,360.0,21.387786,75.425954,39.957252,727.566412,1.535878,6.590840,0.053026,0.479935
123,31418DLT7,0.715344,0.085829,0.062365,0.027478,0.108984,0.871257,0.128743,0.307811,0.389935,0.302254,0.016981,0.949676,0.033344,0.266440,0.326027,0.407533,3.868301,360.0,21.329423,70.568694,37.676443,749.874653,1.512813,5.132140,0.051486,0.469695


In [86]:
data2111_2112 = get_data(fnm2111,cpr2111_2112)
data2111_2112
#data2111_2112.to_csv('data2111_2112.csv')

,CUSIP,CA,FL,IL,NY,TX,No,Yes,Cash-Out,N,Purchase,Investor,Principal,Second,Broker,Correspondent,Retail,Original Interest Rate,Loan Term,Loan Age,Loan-To-Value (LTV),Debt-To-Income (DTI),Borrower Credit Score,Number of Borrowers,Mortgage Insurance Percent,SMM,CPR
0,31418D3X8,0.772589,0.085494,0.042690,0.019227,0.080000,0.866495,0.133505,0.321202,0.420658,0.258140,0.005207,0.976023,0.018770,0.465694,0.187811,0.346495,2.872175,360.0,2.972074,63.303119,36.415050,759.116624,1.475193,3.630329,0.001964,0.023319
1,31418D3Y6,0.698721,0.088416,0.086203,0.036645,0.090015,0.839646,0.160354,0.477004,0.253566,0.269429,0.013527,0.962863,0.023610,0.195401,0.185563,0.619036,3.310796,360.0,3.092474,70.493237,37.462863,737.321323,1.505534,5.036153,0.004349,0.050953
2,31418D4V1,0.375000,0.375000,NaN,NaN,0.250000,1.000000,NaN,0.687500,0.062500,0.250000,0.375000,0.562500,0.062500,0.062500,0.250000,0.687500,4.882812,360.0,3.125000,71.125000,34.437500,696.937500,1.625000,2.312500,0.000746,0.008919
3,31418D4A7,0.365854,0.121951,0.109756,0.048780,0.353659,0.951220,0.048780,0.695122,0.158537,0.146341,0.134146,0.841463,0.024390,0.085366,0.219512,0.695122,4.342683,360.0,3.121951,74.243902,38.658537,689.378049,1.439024,2.878049,0.019154,0.207116
4,31418D3W0,0.813559,0.092010,0.012107,0.012107,0.070218,0.949153,0.050847,0.208232,0.690073,0.101695,NaN,0.987893,0.012107,0.498789,0.278450,0.222760,2.370646,360.0,3.106538,56.617433,34.346247,768.302663,1.472155,1.583535,-0.001075,-0.012972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,31418DLU4,0.563910,0.142857,0.075188,0.056391,0.161654,0.778195,0.221805,0.451128,0.139098,0.409774,0.131579,0.853383,0.015038,0.327068,0.360902,0.312030,4.868432,360.0,22.394737,75.755639,39.947368,701.342105,1.560150,5.078947,0.077244,0.618896
121,31418DL34,0.536585,0.182927,0.097561,0.036585,0.146341,0.817073,0.182927,0.500000,0.073171,0.426829,0.243902,0.756098,NaN,0.256098,0.317073,0.426829,5.337427,360.0,22.487805,78.731707,38.975610,690.609756,1.536585,5.939024,0.046345,0.434158
122,31418DLV2,0.620968,0.133871,0.062903,0.022581,0.159677,0.796774,0.203226,0.425806,0.200000,0.374194,0.070968,0.916129,0.012903,0.190323,0.527419,0.282258,4.399965,360.0,22.383871,75.366129,40.051613,727.359677,1.537097,6.572581,0.057088,0.506086
123,31418DLT7,0.710475,0.087508,0.062459,0.028302,0.111256,0.869876,0.130124,0.310995,0.384190,0.304815,0.017241,0.949577,0.033182,0.266103,0.326610,0.407287,3.868273,360.0,22.330514,70.477228,37.692258,749.771958,1.510085,5.059857,0.041271,0.396958


In [87]:
data2112_2201 = get_data(fnm2112,cpr2112_2201)
data2112_2201
#data2112_2201.to_csv('data2112_2201.csv')

,CUSIP,CA,FL,IL,NY,TX,No,Yes,Cash-Out,N,Purchase,Investor,Principal,Second,Broker,Correspondent,Retail,Original Interest Rate,Loan Term,Loan Age,Loan-To-Value (LTV),Debt-To-Income (DTI),Borrower Credit Score,Number of Borrowers,Mortgage Insurance Percent,SMM,CPR
0,31418D3X8,0.772420,0.085550,0.042718,0.019266,0.080046,0.866342,0.133658,0.320986,0.420700,0.258314,0.005218,0.975975,0.018807,0.465826,0.187844,0.346330,2.872132,360.0,3.972248,63.304759,36.419151,759.120929,1.475516,3.628612,0.003113,0.036717
1,31418D3Y6,0.697703,0.088787,0.086565,0.036799,0.090146,0.839220,0.160780,0.477031,0.253519,0.269449,0.013584,0.962954,0.023463,0.194739,0.185601,0.619659,3.310788,360.0,4.092122,70.490738,37.462213,737.346505,1.505927,5.046802,0.006931,0.080075
2,31418D4V1,0.375000,0.375000,NaN,NaN,0.250000,1.000000,NaN,0.687500,0.062500,0.250000,0.375000,0.562500,0.062500,0.062500,0.250000,0.687500,4.882812,360.0,4.125000,71.125000,34.437500,696.937500,1.625000,2.312500,-0.000525,-0.006321
3,31418D4A7,0.370370,0.111111,0.111111,0.049383,0.358025,0.950617,0.049383,0.703704,0.160494,0.135802,0.135802,0.851852,0.012346,0.086420,0.209877,0.703704,4.342284,360.0,4.098765,74.049383,38.592593,687.802469,1.444444,2.604938,0.015429,0.170222
4,31418D3W0,0.815534,0.089806,0.012136,0.012136,0.070388,0.949029,0.050971,0.208738,0.689320,0.101942,NaN,0.987864,0.012136,0.500000,0.276699,0.223301,2.370939,360.0,4.104369,56.563107,34.339806,768.308252,1.473301,1.587379,0.002474,0.029290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,31418DLU4,0.548387,0.153226,0.076613,0.060484,0.161290,0.766129,0.233871,0.451613,0.125000,0.423387,0.129032,0.854839,0.016129,0.330645,0.358871,0.310484,4.870351,360.0,23.403226,75.814516,39.943548,701.399194,1.576613,5.205645,0.071898,0.591542
121,31418DL34,0.518987,0.189873,0.101266,0.037975,0.151899,0.822785,0.177215,0.506329,0.075949,0.417722,0.240506,0.759494,NaN,0.265823,0.291139,0.443038,5.334418,360.0,23.468354,78.594937,38.810127,689.202532,1.531646,5.784810,0.043099,0.410611
122,31418DLV2,0.614601,0.139219,0.062818,0.023769,0.159593,0.799660,0.200340,0.421053,0.198642,0.380306,0.071307,0.915110,0.013582,0.188455,0.529711,0.281834,4.400728,360.0,23.382003,75.431239,40.108659,727.137521,1.531409,6.570458,0.051801,0.471804
123,31418DLT7,0.707060,0.087916,0.062458,0.029192,0.113374,0.868975,0.131025,0.311948,0.381195,0.306857,0.016972,0.951120,0.031908,0.264766,0.325526,0.409708,3.868201,360.0,23.328921,70.425322,37.711473,749.887305,1.510183,5.099457,0.038777,0.377861


In [88]:
data2201_2202 = get_data(fnm2201,cpr2201_2202)
data2201_2202
#data2201_2202.to_csv('data2201_2202.csv')

,CUSIP,CA,FL,IL,NY,TX,No,Yes,Cash-Out,N,Purchase,Investor,Principal,Second,Broker,Correspondent,Retail,Original Interest Rate,Loan Term,Loan Age,Loan-To-Value (LTV),Debt-To-Income (DTI),Borrower Credit Score,Number of Borrowers,Mortgage Insurance Percent,SMM,CPR
0,31418D3X8,0.772201,0.085468,0.042878,0.019338,0.080115,0.866129,0.133871,0.320576,0.420719,0.258705,0.005180,0.976000,0.018820,0.466014,0.187453,0.346532,2.872087,360.0,4.971453,63.306129,36.424921,759.136230,1.475683,3.633727,0.003797,0.044620
1,31418D3Y6,0.696420,0.089135,0.086897,0.036922,0.090627,0.838637,0.161363,0.476131,0.254102,0.269766,0.013426,0.963327,0.023247,0.194182,0.185853,0.619965,3.310229,360.0,5.089135,70.508454,37.459349,737.352188,1.506216,5.063774,0.007016,0.081015
2,31418D4V1,0.375000,0.375000,NaN,NaN,0.250000,1.000000,NaN,0.687500,0.062500,0.250000,0.375000,0.562500,0.062500,0.062500,0.250000,0.687500,4.882812,360.0,5.125000,71.125000,34.437500,696.937500,1.625000,2.312500,-0.000162,-0.001951
3,31418D4A7,0.375000,0.112500,0.100000,0.050000,0.362500,0.950000,0.050000,0.700000,0.162500,0.137500,0.137500,0.850000,0.012500,0.087500,0.200000,0.712500,4.341875,360.0,5.087500,73.975000,38.450000,686.325000,1.450000,2.637500,0.025296,0.264685
4,31418D3W0,0.815085,0.090024,0.012165,0.012165,0.070560,0.948905,0.051095,0.206813,0.690998,0.102190,NaN,0.987835,0.012165,0.498783,0.277372,0.223844,2.370650,360.0,5.107056,56.540146,34.345499,768.304136,1.472019,1.591241,0.003263,0.038464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,31418DLU4,0.563319,0.144105,0.074236,0.061135,0.157205,0.755459,0.244541,0.445415,0.113537,0.441048,0.135371,0.847162,0.017467,0.327511,0.366812,0.305677,4.872996,360.0,24.406114,75.960699,39.550218,700.563319,1.563319,5.135371,0.049407,0.455582
121,31418DL34,0.525641,0.192308,0.089744,0.038462,0.153846,0.833333,0.166667,0.512821,0.076923,0.410256,0.243590,0.756410,NaN,0.269231,0.294872,0.435897,5.335500,360.0,24.474359,78.384615,39.076923,689.025641,1.538462,5.474359,0.031646,0.320152
122,31418DLV2,0.616487,0.141577,0.059140,0.023297,0.159498,0.793907,0.206093,0.428315,0.188172,0.383513,0.071685,0.915771,0.012545,0.191756,0.526882,0.281362,4.398351,360.0,24.378136,75.274194,40.082437,727.112903,1.541219,6.559140,0.042604,0.406943
123,31418DLT7,0.704553,0.087893,0.062478,0.030357,0.114719,0.869749,0.130251,0.315567,0.377691,0.306742,0.017649,0.950935,0.031415,0.263325,0.323685,0.412990,3.867859,360.0,24.331098,70.256618,37.677374,750.155665,1.510413,4.993293,0.032487,0.327203
